# BetFair Dataset Feature Engineering

@author Nathan Stevens<br>
@version 0.2 10/03/2023 

In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import time
from datetime import datetime
import pickle

# load our api keys
from config import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [25]:
# load the main dataset from pickle file
betfair_df = pd.read_pickle("resources/betfair_dataset.pkl")
betfair_df.info()
betfair_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043349 entries, 0 to 1043348
Data columns (total 18 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   SPORTS_ID         1043349 non-null  object 
 1   EVENT_ID          1043349 non-null  object 
 2   SETTLED_DATE      1043349 non-null  object 
 3   FULL_DESCRIPTION  1043349 non-null  object 
 4   SCHEDULED_OFF     1043349 non-null  object 
 5   EVENT             1043349 non-null  object 
 6   DT ACTUAL_OFF     1043349 non-null  object 
 7   SELECTION_ID      1043349 non-null  int64  
 8   SELECTION         1043349 non-null  object 
 9   ODDS              1043349 non-null  float64
 10  NUMBER_BETS       1043349 non-null  int64  
 11  VOLUME_MATCHED    1043349 non-null  float64
 12  LATEST_TAKEN      1043349 non-null  object 
 13  FIRST_TAKEN       1043349 non-null  object 
 14  WIN_FLAG          1043349 non-null  int64  
 15  IN_PLAY           1043349 non-null  object 
 16  

,SPORTS_ID,EVENT_ID,SETTLED_DATE,FULL_DESCRIPTION,SCHEDULED_OFF,EVENT,DT ACTUAL_OFF,SELECTION_ID,SELECTION,ODDS,NUMBER_BETS,VOLUME_MATCHED,LATEST_TAKEN,FIRST_TAKEN,WIN_FLAG,IN_PLAY,WIN_AMOUNT,LOSS_AMOUNT
0,Soccer,115279736,04-09-2014 17:07:30,U21 Euro Championship/Qualifiers/Fixtures 04 S...,04-09-2014 15:30,Over/Under 1.5 Goals,04-09-2014 15:32:39,1221386,Over 1.5 Goals,1.35,7,765.54,04-09-2014 16:39:49,04-09-2014 16:39:28,1,IP,1033.4790,0.0
1,Tennis,115288925,04-09-2014 11:50:25,Group A/Brasov Challenger 2014/Second Round Ma...,04-09-2014 10:30,Match Odds,04-09-2014 10:34:32,7659748,Aslan Karatsev,1.12,41,3836.70,04-09-2014 11:44:26,04-09-2014 11:18:57,1,IP,4297.1040,0.0
2,Soccer,115196118,03-09-2014 15:08:55,U21 Euro Championship/Qualifiers/Fixtures 03 S...,03-09-2014 14:00,Over/Under 1.5 Goals,03-09-2014 14:00:45,1221386,Over 1.5 Goals,2.10,6,625.54,03-09-2014 15:00:37,03-09-2014 14:45:28,1,IP,1313.6340,0.0
3,Soccer,115236324,01-09-2014 19:46:39,Irish Soccer/FAI Ford Cup/Fixtures 01 Septembe...,01-09-2014 19:00,Over/Under 0.5 Goals,01-09-2014 19:01:34,5851483,Over 0.5 Goals,1.10,22,1176.46,01-09-2014 19:26:21,01-09-2014 19:20:00,1,IP,1294.1060,0.0
4,Soccer,115184779,02-09-2014 18:38:14,Swedish Soccer/Superettan/Fixtures 02 Septembe...,02-09-2014 17:20,Over/Under 1.5 Goals,02-09-2014 17:20:13,1221386,Over 1.5 Goals,1.34,19,903.18,02-09-2014 18:30:48,02-09-2014 18:28:48,1,IP,1210.2612,0.0


In [5]:
# define of names of files containing name of spotting event and rough location
sport_files = ["golf_location.csv", "american_football_location.csv", "basketball_location.csv",
              "cricket_location.csv", "baseball_location.csv", "tennis_location.csv", "soccer_location.csv"]

sports = ["Golf", "American Football", "Basketball", "Cricket", "Baseball", "Tennis", "Soccer"]

# read in the first file and display the dataframe
df = pd.read_csv("output/" + sport_files[6], encoding='utf-8')
df

,FULL_DESCRIPTION,NUMBER_BETS,VOLUME_MATCHED,WIN_AMOUNT,LOSS_AMOUNT,SETTLED_DATE,COUNTRY,CITY
0,Algerian Soccer/Ligue 1/Fixtures 06 September/...,1273,16644.00,12232.6506,21270.4386,6/9/14 20:01,Algeria,Algiers
1,Anderlecht (W) v OH Leuven (W)/Asian Handicap,2,7.10,0.0000,19.5250,NaN,Belgium,Leuven
2,Argentinian Soccer/Argentinian Primera B Metro...,1693,30690.96,33057.8786,28235.8778,1/9/14 17:37,Argentina,Buenos Aires
3,Argentinian Soccer/Argentinian Primera B Metro...,2,16.40,57.5640,0.0000,NaN,Argentina,Buenos Aires
4,Argentinian Soccer/Argentinian Primera B Metro...,2953,37557.27,39084.1152,49958.4387,2/9/14 1:57,Argentina,Buenos Aires
...,...,...,...,...,...,...,...,...
1954,Womens Soccer/Women's Super League 2/Fixtures ...,677,12267.28,11040.3920,16842.5044,4/9/14 20:36,United Kingdom,Birmingham
1955,Womens Soccer/Women's Super League 2/Fixtures ...,382,4437.80,5537.3990,1626.6658,7/9/14 15:36,United Kingdom,Birmingham
1956,Womens Soccer/Women's Super League 2/Fixtures ...,307,4297.10,4548.1826,2156.8096,7/9/14 13:59,United Kingdom,Birmingham
1957,Womens Soccer/Women's Super League 2/Fixtures ...,1094,29389.31,28708.7117,33928.8714,7/9/14 13:46,United Kingdom,Birmingham


In [6]:
# convert the date and time to unix time stamp
def get_timestamp(datetime_str):
    datetime_object = datetime.strptime(datetime_str, '%d/%m/%y %H:%M')
    ttuple = datetime_object.timetuple()
    
    timestamp = int(time.mktime(ttuple)) - 18000 # remove 18000 seconds so we match the timezone in betfair dataset
    date = datetime_object.strftime("%Y-%m-%d")
    return(timestamp, date)

In [7]:
# function to get the historical weather using Open-Meteo Historical Weather API
# https://open-meteo.com/en/docs/historical-weather-api
def get_weather(lat, lon, game_datetime):
    # set temp to -1, and rain to 0
    temp = -1
    rain = 0
    weather_url = ""
    
    # debug for when we exceed the max hour api calls
    # return(temp, rain)
    
    try:
        # get the timestamp from the date string
        timestamp, game_date = get_timestamp(game_datetime)
        
        weather_url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={game_date}&end_date={game_date}&hourly=temperature_2m,rain&timeformat=unixtime"
        weather_data = requests.get(weather_url).json()
        
        rains = weather_data["hourly"]["rain"]
        times = weather_data["hourly"]["time"]
        temps = weather_data["hourly"]["temperature_2m"]
              
        # grab the temperature and rain amount within 30 minutes
        for i in range(len(times)):
            tdiff = abs(times[i] - timestamp)
            
            if tdiff < 1800:
                temp = temps[i]
                
                if rains[i] > 0:
                    rain = 1
                    
                #print(times[i], tdiff, temps[i], rain)
                break
        # we didn't find a temp lets just use the max
        if temp == -1:
            temp = max(temps)
            print("Max Temp (C), Rained?", temp, rain)
    except Exception as e:
        print("Unable to get weather data for", lat, lon, game_datetime)
        print(weather_url)
        print("Exception:", e)
        
    return(temp, rain)

In [7]:
# define dictionary to store city locations to avoid hitting the geoapify API every time
city_locations = dict()

In [ ]:
# save the city_locations to pickle file
with open('output/city_location.pkl', 'wb') as handle:
    pickle.dump(city_locations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
# open the city_locations from the pickle file
with open('output/city_location.pkl', 'rb') as handle:
    city_locations = pickle.load(handle)

In [9]:
# function to process a csv file containing country and cities and return lat, lon, temp, rain?
base_url = "https://api.geoapify.com/v1/geocode/search?text="

def get_location(filename):
    #df = pd.read_csv(filename, encoding='ISO-8859-1')
    df = pd.read_csv(filename, encoding='utf-8')
    df["LATITUDE"] = ""
    df["LONGITUDE"] = ""
    df["TEMPERATURE"] = ""
    df["RAIN"] = ""
    
    game_datetime = ""
    
    # process each row
    for index, row in df.iterrows():        
        target_city = row["CITY"].strip() + ", " + row["COUNTRY"].strip()
        endpoint_url = base_url + target_city + "&format=json&apiKey=" + geoapify_key
        
        if type(row["SETTLED_DATE"]) == str:
            game_datetime = row["SETTLED_DATE"]
            
        #print("Game Date:", game_datetime)
        # call the api for taget city or used saved values
        try:
            if target_city in city_locations:
                lat, lon = city_locations[target_city]
                #print("Using save location:", target_city, lat, lon)
            else:
                geo_data = requests.get(endpoint_url).json()
            
                lat = geo_data["results"][0]["lat"]
                lon = geo_data["results"][0]["lon"]
                city_locations[target_city] = (lat, lon)
                #print("Using geoapify API:",target_city, lat,lon)
        except Exception as e:
            lat = -1
            lon = -1
            print("\nNo Latitude/Longitude:", target_city)
            print(endpoint_url)
            print(e)
        
        # get the weather tuple containing temperature, and rain (1=rained, 0=no rain)
        weather = get_weather(lat, lon, game_datetime)
        
        print(filename, (index + 2),"-- Processing City: ", target_city, lat, lon, weather)
        
        # update the dataframe with the lat,lon, temperature and weather
        df.loc[index, "LATITUDE"] = lat
        df.loc[index, "LONGITUDE"] = lon
        df.loc[index, "TEMPERATURE"] = weather[0]
        df.loc[index, "RAIN"] = weather[1]
    # return the modified dataframe
    return df

In [15]:
%%time

# add lat,lon, and weather info to the sports files
sports_df = []

for idx, filename in enumerate(sport_files):
    df = get_location("output/" + filename)
    df.insert(0, "SPORTS_ID", sports[idx])
    
    sports_df.append(df)
    
    # Debug code to only process a small number of the files. 
    # Set to > size of sport_files to process all files
    if idx == 10:
        break
    
print("Total cities:", len(city_locations), "\n")

output/golf_location.csv 2 -- Processing City:  Cherry Hills Village, CO, USA 39.6416544 -104.959425 (16.6, 0)
output/golf_location.csv 3 -- Processing City:  Cherry Hills Village, CO, USA 39.6416544 -104.959425 (27.4, 0)
output/golf_location.csv 4 -- Processing City:  Davidson, NC, USA 35.4992614 -80.8485218 (29.2, 0)
output/golf_location.csv 5 -- Processing City:  Norton, Massachusetts, USA 41.9667666 -71.1869939 (28.3, 0)
output/golf_location.csv 6 -- Processing City:  Norton, Massachusetts, USA 41.9667666 -71.1869939 (28.3, 0)
output/golf_location.csv 7 -- Processing City:  Crans-Montana, Switzerland 46.3304899 7.526435803722132 (17.1, 0)
output/golf_location.csv 8 -- Processing City:  Crans-Montana, Switzerland 46.3304899 7.526435803722132 (11.2, 0)
output/golf_location.csv 9 -- Processing City:  Portland, Oregon, USA 45.5202471 -122.674194 (23.3, 0)
output/golf_location.csv 10 -- Processing City:  Levis, Quebec, Canada 46.8033114 -71.1778792 (19.0, 0)
output/american_football_loc

output/basketball_location.csv 18 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (33.9, 0)
output/basketball_location.csv 19 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (33.9, 0)
output/basketball_location.csv 20 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (32.1, 0)
output/basketball_location.csv 21 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (33.9, 0)
output/basketball_location.csv 22 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (35.8, 0)
output/basketball_location.csv 23 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (29.0, 0)
output/basketball_location.csv 24 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (32.0, 0)
output/basketball_location.csv 25 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (35.8, 0)
output/basketball_location.csv 26 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (35.6, 0)
output/basketball_location.csv 27 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 

output/cricket_location.csv 32 -- Processing City:  Kent, United Kingdom 51.2474823 0.7105077 (19.6, 1)
output/cricket_location.csv 33 -- Processing City:  Durham, United Kingdom 54.7770139 -1.5756205 (14.4, 0)
output/cricket_location.csv 34 -- Processing City:  Durham, United Kingdom 54.7770139 -1.5756205 (14.4, 0)
output/cricket_location.csv 35 -- Processing City:  Durham, United Kingdom 54.7770139 -1.5756205 (14.4, 0)
output/cricket_location.csv 36 -- Processing City:  Warwickshire, United Kingdom 52.32130635 -1.5536905536661392 (18.1, 0)
output/cricket_location.csv 37 -- Processing City:  Warwickshire, United Kingdom 52.32130635 -1.5536905536661392 (18.1, 0)
output/cricket_location.csv 38 -- Processing City:  Warwickshire, United Kingdom 52.32130635 -1.5536905536661392 (18.1, 0)
output/cricket_location.csv 39 -- Processing City:  Warwickshire, United Kingdom 52.32130635 -1.5536905536661392 (18.1, 0)
output/cricket_location.csv 40 -- Processing City:  Centurion, South Africa -25.836

output/baseball_location.csv 46 -- Processing City:  Baltimore, Maryland, USA 39.2908816 -76.610759 (24.4, 0)
output/baseball_location.csv 47 -- Processing City:  Cleveland, Ohio, USA 41.4996574 -81.6936772 (23.8, 0)
output/baseball_location.csv 48 -- Processing City:  Minneapolis, Minnesota, USA 44.9772995 -93.2654692 (24.0, 1)
output/baseball_location.csv 49 -- Processing City:  Arlington, Texas, USA 32.7355816 -97.1071186 (31.2, 0)
output/baseball_location.csv 50 -- Processing City:  Milwaukee, Wisconsin, USA 43.0349931 -87.922497 (23.8, 0)
output/baseball_location.csv 51 -- Processing City:  St. Petersburg, Florida, USA 27.7700475 -82.6359078 (27.5, 0)
output/baseball_location.csv 52 -- Processing City:  Los Angeles, California, USA 34.0536909 -118.242766 (24.4, 0)
output/baseball_location.csv 53 -- Processing City:  Miami, Florida, USA 25.7741728 -80.19362 (26.4, 0)
output/baseball_location.csv 54 -- Processing City:  St. Petersburg, Florida, USA 27.7700475 -82.6359078 (27.9, 0)
o

output/tennis_location.csv 23 -- Processing City:  Alphen aan den Rijn, Netherlands 52.1131403 4.6408411999450685 (21.4, 0)
output/tennis_location.csv 24 -- Processing City:  Alphen aan den Rijn, Netherlands 52.1131403 4.6408411999450685 (21.5, 0)
output/tennis_location.csv 25 -- Processing City:  Alphen aan den Rijn, Netherlands 52.1131403 4.6408411999450685 (21.4, 0)
output/tennis_location.csv 26 -- Processing City:  Alphen aan den Rijn, Netherlands 52.1131403 4.6408411999450685 (20.7, 0)
output/tennis_location.csv 27 -- Processing City:  Alphen aan den Rijn, Netherlands 52.1131403 4.6408411999450685 (21.4, 0)
output/tennis_location.csv 28 -- Processing City:  Alphen aan den Rijn, Netherlands 52.1131403 4.6408411999450685 (19.9, 0)
output/tennis_location.csv 29 -- Processing City:  Alphen aan den Rijn, Netherlands 52.1131403 4.6408411999450685 (21.4, 0)
output/tennis_location.csv 30 -- Processing City:  Alphen aan den Rijn, Netherlands 52.1131403 4.6408411999450685 (19.0, 0)
output/t

...

output/soccer_location.csv 1575 -- Processing City:  Bratislava, Slovakia 48.1516988 17.1093063 (21.5, 1)
output/soccer_location.csv 1576 -- Processing City:  Bratislava, Slovakia 48.1516988 17.1093063 (22.5, 0)
output/soccer_location.csv 1577 -- Processing City:  Bratislava, Slovakia 48.1516988 17.1093063 (22.5, 0)
output/soccer_location.csv 1578 -- Processing City:  Bratislava, Slovakia 48.1516988 17.1093063 (23.0, 0)
output/soccer_location.csv 1579 -- Processing City:  Bratislava, Slovakia 48.1516988 17.1093063 (23.0, 0)
output/soccer_location.csv 1580 -- Processing City:  Bratislava, Slovakia 48.1516988 17.1093063 (22.5, 0)
output/soccer_location.csv 1581 -- Processing City:  Bratislava, Slovakia 48.1516988 17.1093063 (23.1, 0)
output/soccer_location.csv 1582 -- Processing City:  Bratislava, Slovakia 48.1516988 17.1093063 (23.1, 0)
output/soccer_location.csv 1583 -- Processing City:  Bratislava, Slovakia 48.1516988 17.1093063 (21.9, 0)
output/soccer_location.csv 1584 -- Processing 

output/soccer_location.csv 1657 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (30.4, 0)
output/soccer_location.csv 1658 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (30.4, 0)
output/soccer_location.csv 1659 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (25.7, 0)
output/soccer_location.csv 1660 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (25.7, 0)
output/soccer_location.csv 1661 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (25.5, 0)
output/soccer_location.csv 1662 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (27.2, 0)
output/soccer_location.csv 1663 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (28.4, 0)
output/soccer_location.csv 1664 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (27.2, 0)
output/soccer_location.csv 1665 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (27.2, 0)
output/soccer_location.csv 1666 -- Processing City:  Madrid, Spain 40.4167047 -3.7035825 (25.5, 0)
output/soc

output/soccer_location.csv 1740 -- Processing City:  Stockholm, Sweden 59.3251172 18.0710935 (15.3, 0)
output/soccer_location.csv 1741 -- Processing City:  Stockholm, Sweden 59.3251172 18.0710935 (15.3, 0)
output/soccer_location.csv 1742 -- Processing City:  Stockholm, Sweden 59.3251172 18.0710935 (16.8, 0)
output/soccer_location.csv 1743 -- Processing City:  Stockholm, Sweden 59.3251172 18.0710935 (16.8, 0)
output/soccer_location.csv 1744 -- Processing City:  Stockholm, Sweden 59.3251172 18.0710935 (16.8, 0)
output/soccer_location.csv 1745 -- Processing City:  Stockholm, Sweden 59.3251172 18.0710935 (13.3, 0)
output/soccer_location.csv 1746 -- Processing City:  Stockholm, Sweden 59.3251172 18.0710935 (13.3, 0)
output/soccer_location.csv 1747 -- Processing City:  Stockholm, Sweden 59.3251172 18.0710935 (13.3, 0)
output/soccer_location.csv 1748 -- Processing City:  Stockholm, Sweden 59.3251172 18.0710935 (17.7, 0)
output/soccer_location.csv 1749 -- Processing City:  Stockholm, Sweden 59

output/soccer_location.csv 1820 -- Processing City:  Oslo, Norway 59.9133301 10.7389701 (17.4, 0)
output/soccer_location.csv 1821 -- Processing City:  Bucharest, Romania 44.4361414 26.1027202 (24.8, 0)
output/soccer_location.csv 1822 -- Processing City:  Bucharest, Romania 44.4361414 26.1027202 (24.8, 0)
output/soccer_location.csv 1823 -- Processing City:  Bucharest, Romania 44.4361414 26.1027202 (24.8, 0)
output/soccer_location.csv 1824 -- Processing City:  Kyiv, Ukraine 50.4500336 30.5241361 (17.1, 0)
output/soccer_location.csv 1825 -- Processing City:  Kyiv, Ukraine 50.4500336 30.5241361 (17.1, 0)
output/soccer_location.csv 1826 -- Processing City:  Kyiv, Ukraine 50.4500336 30.5241361 (17.1, 0)
output/soccer_location.csv 1827 -- Processing City:  Nicosia, Cyprus 35.1748976 33.3638568 (27.0, 0)
output/soccer_location.csv 1828 -- Processing City:  Rome, Italy 41.8933203 12.4829321 (22.8, 0)
output/soccer_location.csv 1829 -- Processing City:  Vienna, Austria 48.2083537 16.3725042 (20.

output/soccer_location.csv 1901 -- Processing City:  Cardiff, Wales 51.4816546 -3.1791934 (19.3, 1)
output/soccer_location.csv 1902 -- Processing City:  Cardiff, Wales 51.4816546 -3.1791934 (19.3, 1)
output/soccer_location.csv 1903 -- Processing City:  Cardiff, Wales 51.4816546 -3.1791934 (17.7, 0)
output/soccer_location.csv 1904 -- Processing City:  Cardiff, Wales 51.4816546 -3.1791934 (17.7, 0)
output/soccer_location.csv 1905 -- Processing City:  Cardiff, Wales 51.4816546 -3.1791934 (17.7, 0)
output/soccer_location.csv 1906 -- Processing City:  Cardiff, Wales 51.4816546 -3.1791934 (17.7, 0)
output/soccer_location.csv 1907 -- Processing City:  Brussels, Belgium 50.8465573 4.351697 (18.9, 0)
output/soccer_location.csv 1908 -- Processing City:  Brussels, Belgium 50.8465573 4.351697 (18.9, 0)
output/soccer_location.csv 1909 -- Processing City:  Brussels, Belgium 50.8465573 4.351697 (17.3, 0)
output/soccer_location.csv 1910 -- Processing City:  Brussels, Belgium 50.8465573 4.351697 (17.3,

In [26]:
# concat and save the sports all dataframe
sports_all_df = pd.concat(sports_df, axis=0, ignore_index=True)

# save to scv file
sports_all_df.to_csv('output/sports_location.csv', index=False)
sports_all_df.head()

,SPORTS_ID,FULL_DESCRIPTION,NUMBER_BETS,VOLUME_MATCHED,WIN_AMOUNT,LOSS_AMOUNT,SETTLED_DATE,COUNTRY,CITY,LATITUDE,LONGITUDE,TEMPERATURE,RAIN
0,Golf,Group B/BMW Championship 2014,149070,4712927.68,5.190166e+06,8.088417e+07,5/9/14 17:40,USA,"Cherry Hills Village, CO",39.641654,-104.959425,16.6,0
1,Golf,Group B/BMW Championship 2014/Tournament Match...,152,3633.26,6.218803e+03,5.955442e+02,7/9/14 22:20,USA,"Cherry Hills Village, CO",39.641654,-104.959425,27.4,0
2,Golf,Group B/Chiquita Classic 2014,1321,6892.82,4.048800e+02,6.530881e+05,7/9/14 20:11,USA,"Davidson, NC",35.499261,-80.848522,29.2,0
3,Golf,Group B/Deutsche Bank Championship 2014,166663,4672132.07,3.546043e+06,1.027464e+08,1/9/14 21:53,USA,"Norton, Massachusetts",41.966767,-71.186994,28.3,0
4,Golf,Group B/Deutsche Bank Championship 2014/Tourna...,415,10449.26,1.569752e+04,3.782393e+03,1/9/14 22:08,USA,"Norton, Massachusetts",41.966767,-71.186994,28.3,0


In [2]:
#load the sports all dataframe
sports_all_df = pd.read_csv("output/sports_location.csv")
sports_all_df

,SPORTS_ID,FULL_DESCRIPTION,NUMBER_BETS,VOLUME_MATCHED,WIN_AMOUNT,LOSS_AMOUNT,SETTLED_DATE,COUNTRY,CITY,LATITUDE,LONGITUDE,TEMPERATURE,RAIN
0,Golf,Group B/BMW Championship 2014,149070,4712927.68,5.190166e+06,8.088417e+07,5/9/14 17:40,USA,"Cherry Hills Village, CO",39.641654,-104.959425,16.6,0
1,Golf,Group B/BMW Championship 2014/Tournament Match...,152,3633.26,6.218803e+03,5.955442e+02,7/9/14 22:20,USA,"Cherry Hills Village, CO",39.641654,-104.959425,27.4,0
2,Golf,Group B/Chiquita Classic 2014,1321,6892.82,4.048800e+02,6.530881e+05,7/9/14 20:11,USA,"Davidson, NC",35.499261,-80.848522,29.2,0
3,Golf,Group B/Deutsche Bank Championship 2014,166663,4672132.07,3.546043e+06,1.027464e+08,1/9/14 21:53,USA,"Norton, Massachusetts",41.966767,-71.186994,28.3,0
4,Golf,Group B/Deutsche Bank Championship 2014/Tourna...,415,10449.26,1.569752e+04,3.782393e+03,1/9/14 22:08,USA,"Norton, Massachusetts",41.966767,-71.186994,28.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2653,Soccer,Womens Soccer/Women's Super League 2/Fixtures ...,677,12267.28,1.104039e+04,1.684250e+04,4/9/14 20:36,United Kingdom,Birmingham,52.479699,-1.902691,17.0,0
2654,Soccer,Womens Soccer/Women's Super League 2/Fixtures ...,382,4437.80,5.537399e+03,1.626666e+03,7/9/14 15:36,United Kingdom,Birmingham,52.479699,-1.902691,17.6,0
2655,Soccer,Womens Soccer/Women's Super League 2/Fixtures ...,307,4297.10,4.548183e+03,2.156810e+03,7/9/14 13:59,United Kingdom,Birmingham,52.479699,-1.902691,17.2,0
2656,Soccer,Womens Soccer/Women's Super League 2/Fixtures ...,1094,29389.31,2.870871e+04,3.392887e+04,7/9/14 13:46,United Kingdom,Birmingham,52.479699,-1.902691,17.2,0


In [28]:
sports_betfair_df = betfair_df.loc[betfair_df["SPORTS_ID"].isin(sports)]
sports_betfair_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1011027 entries, 0 to 1043348
Data columns (total 18 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   SPORTS_ID         1011027 non-null  object 
 1   EVENT_ID          1011027 non-null  object 
 2   SETTLED_DATE      1011027 non-null  object 
 3   FULL_DESCRIPTION  1011027 non-null  object 
 4   SCHEDULED_OFF     1011027 non-null  object 
 5   EVENT             1011027 non-null  object 
 6   DT ACTUAL_OFF     1011027 non-null  object 
 7   SELECTION_ID      1011027 non-null  int64  
 8   SELECTION         1011027 non-null  object 
 9   ODDS              1011027 non-null  float64
 10  NUMBER_BETS       1011027 non-null  int64  
 11  VOLUME_MATCHED    1011027 non-null  float64
 12  LATEST_TAKEN      1011027 non-null  object 
 13  FIRST_TAKEN       1011027 non-null  object 
 14  WIN_FLAG          1011027 non-null  int64  
 15  IN_PLAY           1011027 non-null  object 
 16  WIN_A

In [22]:
#add the new columns
sports_betfair_df["COUNTRY"] = ""
sports_betfair_df["CITY"] = ""
sports_betfair_df["LATITUDE"] = ""
sports_betfair_df["LONGITUDE"] = ""
sports_betfair_df["TEMPERATURE"] = ""
sports_betfair_df["RAIN"] = ""

In [11]:
%%time

# use apply to update data frame
def add_location_weather(row):
    description = row["FULL_DESCRIPTION"]
    
    sport_row = sports_all_df.loc[sports_all_df["FULL_DESCRIPTION"] == description]
    
    if sport_row.empty:
        print("No data for", description)
    else:
        row["COUNTRY"] = sport_row["COUNTRY"].iloc[0]
        row["CITY"] = sport_row["CITY"].iloc[0]
        row["LATITUDE"] = sport_row["LATITUDE"].iloc[0]
        row["LONGITUDE"] = sport_row["LONGITUDE"].iloc[0]
        row["TEMPERATURE"] = sport_row["TEMPERATURE"].iloc[0]
        row["RAIN"] = sport_row["RAIN"].iloc[0]
    
    return row

# use apply to update row information
sports_betfair_df = sports_betfair_df.apply(add_location_weather, axis=1)

CPU times: user 4min 43s, sys: 6 s, total: 4min 49s
Wall time: 4min 45s


In [29]:
%%time

# process each row in the filtered betfair dataframe to add in the locations temperature data. 
for index, row in sports_betfair_df.iterrows():
    description = row["FULL_DESCRIPTION"]
    
    sport_row = sports_all_df.loc[sports_all_df["FULL_DESCRIPTION"] == description]
    
    if sport_row.empty:
        print("No data for", description)
    else:
        sports_betfair_df.loc[index, "COUNTRY"] = sport_row["COUNTRY"].iloc[0]
        sports_betfair_df.loc[index, "CITY"] = sport_row["CITY"].iloc[0]
        sports_betfair_df.loc[index, "LATITUDE"] = sport_row["LATITUDE"].iloc[0]
        sports_betfair_df.loc[index, "LONGITUDE"] = sport_row["LONGITUDE"].iloc[0]
        sports_betfair_df.loc[index, "TEMPERATURE"] = sport_row["TEMPERATURE"].iloc[0]
        sports_betfair_df.loc[index, "RAIN"] = sport_row["RAIN"].iloc[0]
    
    if index % 25000 == 0:
        print("Processing Row:", index)
# print blank line
print("")

Processing Row: 0
Processing Row: 25000
Processing Row: 50000
Processing Row: 75000
Processing Row: 100000
Processing Row: 125000
Processing Row: 150000
Processing Row: 175000
Processing Row: 200000
Processing Row: 225000
Processing Row: 250000
Processing Row: 275000
Processing Row: 300000
Processing Row: 325000
Processing Row: 350000
Processing Row: 400000
Processing Row: 425000
Processing Row: 450000
Processing Row: 475000
Processing Row: 500000
Processing Row: 525000
Processing Row: 550000
Processing Row: 575000
Processing Row: 625000
Processing Row: 650000
Processing Row: 675000
Processing Row: 700000
Processing Row: 725000
Processing Row: 750000
Processing Row: 775000
Processing Row: 800000
Processing Row: 825000
Processing Row: 850000
Processing Row: 875000
Processing Row: 900000
Processing Row: 925000
Processing Row: 950000
Processing Row: 975000
Processing Row: 1000000
Processing Row: 1025000

CPU times: user 7min 27s, sys: 17.5 s, total: 7min 44s
Wall time: 7min 33s


In [30]:
sports_betfair_df.head()

,SPORTS_ID,EVENT_ID,SETTLED_DATE,FULL_DESCRIPTION,SCHEDULED_OFF,EVENT,DT ACTUAL_OFF,SELECTION_ID,SELECTION,ODDS,...,WIN_FLAG,IN_PLAY,WIN_AMOUNT,LOSS_AMOUNT,COUNTRY,CITY,LATITUDE,LONGITUDE,TEMPERATURE,RAIN
0,Soccer,115279736,04-09-2014 17:07:30,U21 Euro Championship/Qualifiers/Fixtures 04 S...,04-09-2014 15:30,Over/Under 1.5 Goals,04-09-2014 15:32:39,1221386,Over 1.5 Goals,1.35,...,1,IP,1033.4790,0.0,Ukraine,Kyiv,50.450034,30.524136,17.1,0.0
1,Tennis,115288925,04-09-2014 11:50:25,Group A/Brasov Challenger 2014/Second Round Ma...,04-09-2014 10:30,Match Odds,04-09-2014 10:34:32,7659748,Aslan Karatsev,1.12,...,1,IP,4297.1040,0.0,Romania,Brasov,45.652510,25.610565,22.1,0.0
2,Soccer,115196118,03-09-2014 15:08:55,U21 Euro Championship/Qualifiers/Fixtures 03 S...,03-09-2014 14:00,Over/Under 1.5 Goals,03-09-2014 14:00:45,1221386,Over 1.5 Goals,2.10,...,1,IP,1313.6340,0.0,Latvia,Riga,56.949398,24.105185,17.4,0.0
3,Soccer,115236324,01-09-2014 19:46:39,Irish Soccer/FAI Ford Cup/Fixtures 01 Septembe...,01-09-2014 19:00,Over/Under 0.5 Goals,01-09-2014 19:01:34,5851483,Over 0.5 Goals,1.10,...,1,IP,1294.1060,0.0,Ireland,Dublin,53.349379,-6.260559,16.1,0.0
4,Soccer,115184779,02-09-2014 18:38:14,Swedish Soccer/Superettan/Fixtures 02 Septembe...,02-09-2014 17:20,Over/Under 1.5 Goals,02-09-2014 17:20:13,1221386,Over 1.5 Goals,1.34,...,1,IP,1210.2612,0.0,Sweden,Stockholm,59.325117,18.071093,15.3,0.0


In [31]:
# export to csv and pickle file
sports_betfair_df.to_csv("resources/betfair_final.csv", index=False)
sports_betfair_df.to_pickle("resources/betfair_final.pkl")

In [15]:
sports_betfair_df["RAIN"].value_counts()

RAIN
0    905854
1    105173
Name: count, dtype: int64